In [ ]:
import numpy as np
import glob
import os
import json
import random
import re

In [ ]:
ALL_medical_term = set()
with open('ALL_medical_term_file_Chinese.txt', 'r', encoding='utf8') as f:
	custom_noun = f.readlines()
	for i in range(len(custom_noun)):
		ALL_medical_term.add(custom_noun[i].replace('\n', ''))
ALL_medical_term = list(ALL_medical_term)

In [ ]:
# negative set

path_train, path_base = "train_MDS.txt", "Negative/"
with open(path_train, 'r') as input:
    for jsonObj in input:
        patientDict = json.loads(jsonObj)
        path_id = path_base + str(patientDict["id"]) + "/"
        if not os.path.exists(path_id): os.makedirs(path_id)
        text_list = []
        for i in range(len(patientDict["content"])):
            text_list.append(patientDict["content"][i]["utterance"])
        
        #1: the reference summary itself is untruthful
        if len(patientDict["1_medical"]) > 0 or "泌尿外科男科" in patientDict["summary"]["type-B"] or "关注" in patientDict["summary"]["type-B"]:
            with open(path_id + "negative1.txt", 'w') as test_file:
                test_file.write(patientDict["summary"]["type-B"])
                
        #2: change 1 truthful medical term (the 1st one in "2_medical") to another untruthful one
        if len(patientDict["2_medical"]) > 0:
            truthful_medical_list, random_idx = patientDict["2_medical"] + patientDict["1_medical"], random.randint(0, 2106)
            while ALL_medical_term[random_idx] in truthful_medical_list:
                random_idx = random.randint(0, 2106)

            modified_summary = patientDict["summary"]["type-B"].replace(truthful_medical_list[0], ALL_medical_term[random_idx])
            with open(path_id + "negative2.txt", 'w') as test_file:
                test_file.write(modified_summary)
        
        #3: append an untruthful medical term to the start of the reference summary
        truthful_medical_list, random_idx = patientDict["2_medical"] + patientDict["1_medical"], random.randint(0, 2106)
        while ALL_medical_term[random_idx] in truthful_medical_list:
            random_idx = random.randint(0, 2106)
        modified_summary = ALL_medical_term[random_idx] + patientDict["summary"]["type-B"]
        with open(path_id + "negative3.txt", 'w') as test_file:
            test_file.write(modified_summary)
            
        #4: append an untruthful medical term to the END of the reference summary
#         prev_idx = random_idx
#         random_idx = random.randint(0, 2106)
#         while prev_idx == random_idx or ALL_medical_term[random_idx] in truthful_medical_list:
#             random_idx = random.randint(0, 2106)
#         modified_summary = patientDict["summary"]["type-B"] + ALL_medical_term[random_idx]
#         with open(path_id + "negative4.txt", 'w') as test_file:
#             test_file.write(modified_summary)
            
        #5: copy an utterance and change its numerical value (last numerical value)
        for i in range(len(text_list)-1, -1, -1):
            if len(re.findall(r'\d+', text_list[i])) < 1:
                continue
            num = re.findall(r'\d+', text_list[i])[0]

            random_idx = random.randint(1, 99)
            while random_idx == int(num):
                random_idx = random.randint(1, 99)
            modified_summary = text_list[i].replace(num, str(random_idx))
            
            with open(path_id + "negative5.txt", 'w') as test_file:
                test_file.write(modified_summary)
                break
        
        #6: Negation
        if len(patientDict["2_medical"]) > 0:
            target_term = patientDict["2_medical"][-1]
            for i in range(len(patientDict["content"])-1, 0, -1):
                if target_term in patientDict["content"][i]["utterance"]:
                    if "可以" in patientDict["content"][i]["utterance"] or "不可以" in patientDict["content"][i]["utterance"]:
                        with open(path_id + "negative6.txt", 'w') as test_file:
                            if "可以" in patientDict["content"][i]["utterance"]:
                                negated = patientDict["content"][i]["utterance"].replace("可以", "不可以")
                                test_file.write(negated)
                            elif "不可以" in patientDict["content"][i]["utterance"]:
                                negated = patientDict["content"][i]["utterance"].replace("不可以", "可以")
                    break
                

In [ ]:
# positive set

path_train, path_base = "train_MDS.txt", "Positive/"
with open(path_train, 'r') as input:
    for jsonObj in input:
        patientDict = json.loads(jsonObj)
        path_id = path_base + str(patientDict["id"]) + "/"
        if not os.path.exists(path_id): os.makedirs(path_id)
            
        #1: the reference summary itself is truthful
        if "泌尿外科男科" not in patientDict["summary"]["type-B"] and len(patientDict["1_medical"]) == 0 and "关注" not in patientDict["summary"]["type-B"]:
            with open(path_id + "positive1.txt", 'w') as test_file:
                test_file.write(patientDict["summary"]["type-B"])
        
        #2: extract another utterance with using the last medical term (the last one in "2_medical")
        if len(patientDict["2_medical"]) > 0:
            target_term = patientDict["2_medical"][-1]
            for i in range(len(patientDict["content"])-1, 0, -1):
                if target_term in patientDict["content"][i]["utterance"]:
                    with open(path_id + "positive2.txt", 'w') as test_file:
                        test_file.write(patientDict["content"][i]["utterance"])
                    break
        
#3: extract the last utterance for any diagloue without positive1 and positive2
setOfEmptyDirs = set()
for (dirpath, dirnames, filenames) in os.walk('Positive'):
    if len(dirnames) == 0 and len(filenames) == 0:
        setOfEmptyDirs.add(dirpath.split("/")[-1])

with open(path_train, 'r') as input:
    for jsonObj in input:
        patientDict = json.loads(jsonObj)
        path_id = path_base + str(patientDict["id"]) + "/"
        if str(patientDict["id"]) in setOfEmptyDirs:
            for i in range(len(patientDict["content"])-1, 0, -1):
                if patientDict["content"][i]["utterance"] != patientDict["summary"]["type-B"] and patientDict["content"][i]["speaker"] == "医生":
                    if "关注" in patientDict["content"][i]["utterance"]:
                        continue
                    if "早日康复" in patientDict["content"][i]["utterance"]:
                        continue
                    with open(path_id + "positive3.txt", 'w') as test_file:
                        test_file.write(patientDict["content"][i]["utterance"])
                    break

#4: type-A (description)
with open(path_train, 'r') as input:
    for jsonObj in input:
        patientDict = json.loads(jsonObj)
        path_id = path_base + str(patientDict["id"]) + "/"
            
        with open(path_id + "positive4.txt", 'w') as test_file:
            test_file.write(patientDict["summary"]["description"])